In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Flatten, Concatenate
from gee_scripts.parameters import explain_vars, response_var
import pandas as pd

In [5]:
df = pd.read_csv("data/9_clean_training_data/clean_training_data.csv")

In [1]:
# LSTM Sequence Input
sequence_input_1 = Input(shape=(4, 1), name='sequence_input')  # 4 time steps, 1 feature (sm value)
lstm_out_1 = LSTM(50)(sequence_input_1)  # 50 LSTM units, can be tuned

sequence_input_2 = Input(shape=(4, 1), name='sequence_input')  # 4 time steps, 1 feature (sm value)
lstm_out_2 = LSTM(50)(sequence_input_2)  # 50 LSTM units, can be tuned

# Dense Input for non-sequential data
dense_input = Input(shape=(26,), name='dense_input')  # 26 other explanatory variables
dense_out = Dense(50, activation='relu')(dense_input)  # 50 units, can be tuned

# Combine LSTM and Dense outputs
merged = Concatenate()([lstm_out, dense_out])

# Add further dense layers if needed
dense_merged_1 = Dense(100, activation='relu')(merged)
dense_merged_2 = Dense(50, activation='relu')(dense_merged_1)

# Regression output
output = Dense(1, activation='linear')(dense_merged_2)

# Compile the model
model = Model(inputs=[sequence_input, dense_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Using MSE loss for regression

# Print model summary to see the architecture
model.summary()


2023-11-02 10:35:23.654105: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 10:35:23.714462: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 10:35:23.715456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 10:35:24.826464: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-02 10:35:26.028849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 4, 1)]       0           []                               
                                                                                                  
 dense_input (InputLayer)       [(None, 26)]         0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 50)           10400       ['sequence_input[0][0]']         
                                                                                                  
 dense (Dense)                  (None, 50)           1350        ['dense_input[0][0]']            
                                                                                              

2023-11-02 10:35:26.307564: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-02 10:35:26.309871: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-02 10:35:26.311758: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [10]:
sm_vars = ["sm_3", "sm_7", "sm_30"]
pr_vars = ["prec_3", "prec_7", "prec_30"]
dense_vars = [exp for exp in explain_vars if exp not in sm_vars + pr_vars]

sm_sequence_data = df[sm_vars].values.reshape(-1, 3, 1)
pr_sequence_data = df[pr_vars].values.reshape(-1, 3, 1)
dense_data = df[dense_vars].values.reshape(-1, len(dense_vars))

In [ ]:
model.fit([sm_sequence_data, pr_sequence_data, dense_data], target_values, epochs=10, batch_size=32)

In [ ]:
# Dummy data with 1000 samples and 30 features
X = df[explain_vars].values
y = df[response_var].values